# Distributed SQL with Dask
[Read on Medium](https://blog.blazingdb.com/distributed-sql-with-dask-2979262acc8a?source=friends_link&sk=077319064cd7d9e18df8c0292eb5d33d) | [Launch BlazingSQL Notebooks](https://app.blazingsql.com/jupyter/user-redirect/lab/workspaces/auto-b/tree/Welcome_to_BlazingSQL_Notebooks/blog_posts/distributed_sql_with_dask.ipynb)

Distributed SQL query execution is key in scaling the power of the RAPIDS community to the enterprise-level data challenges of today’s market.

BlazingSQL makes it easy by utilizing Dask and Dask-cuDF (dask_cudf), which means you can effortlessly conduct multiple GPUs through an intuitive Python API.

The easiest way to go distribute BlazingSQL is by leveraging the Dask CUDA project’s `LocalCUDACluster()` and Dask.distributed’s `Client()`, which are both downloaded with BlazingSQL by default (if you have BlazingSQL installed, then you have everything you need).

Just import these packages alongside BlazingSQL, identify your cluster, and pass it’s client to the BlazingContext API upon initialization.

In [ ]:
from dask_cuda import LocalCUDACluster
from dask.distributed import Client

cluster = LocalCUDACluster()
client = Client(cluster)

In [ ]:
from blazingsql import BlazingContext

bc = BlazingContext(dask_client=client, network_interface='lo')

### Create Table
For this demo we’ll use BlazingSQL Distributed to preprocess 24 months of NYC Yellow Cab data, 342M rides in total, and handoff the results to Datashader for near-instant visualization of drop-off coordinates. Our data is stored as 24 Parquet files in a public AWS S3 bucket. Let’s register that bucket and create a table (`taxi`) to hold all our data.

In [ ]:
bc.s3('bsql', bucket_name='blazingsql-colab')

In [ ]:
files_10 = [f's3://bsql/taxi_parquet/yellow_tripdata_2010-0{i}.parquet' if i < 10 
              else f's3://bsql/taxi_parquet/yellow_tripdata_2010-{i}.parquet' for i in range(1, 13)]

files_11 = [f's3://bsql/taxi_parquet/yellow_tripdata_2011-0{i}.parquet' if i < 10 
              else f's3://bsql/taxi_parquet/yellow_tripdata_2011-{i}.parquet' for i in range(1, 13)]

f = files_10 + files_11

In [ ]:
bc.create_table('taxi', f)

### Query a Table & Handoff Results

Let’s define a query that will:
1. Filter data to rides within specified drop-off latitude & longitude coordinates
2. Convert latitude & longitude values to meters as expected by Datashader


In [ ]:
pi = 3.141592653589793
o_shift = 2 * pi * 6378137 / 2

query = f'''
        SELECT 
            dropoff_longitude * {o_shift} / 180 AS dropoff_x, 
            LOG10(TAN(((90 + dropoff_latitude) * {pi} / 360))) / {pi} / 180 * {o_shift} / 180 AS dropoff_y
        FROM 
            taxi
            WHERE
                pickup_longitude < -73.75 AND pickup_longitude > -74.15
                AND dropoff_longitude < -73.75 AND dropoff_longitude > -74.15
                AND pickup_latitude > 40.68 AND pickup_latitude < 40.84
                AND dropoff_latitude > 40.68 AND dropoff_latitude < 40.84
                '''

And we’re ready to visualize. Thanks to Datashader’s support for cuDF and Dask-cuDF, we can plug `bc.sql()` directly in to `.points()`, and pass that `agg` to `.shade()` for near-instant visualization.

In [ ]:
from colorcet import fire
from datashader import Canvas, transfer_functions as tf

In [ ]:
agg = Canvas().points(bc.sql(query), 'dropoff_x', 'dropoff_y')

tf.set_background(tf.shade(agg, cmap=fire), "black")